In [3]:
# ===============================
# Step 1: Install Dependencies
# ===============================
!pip install music21 pretty_midi tensorflow keras tqdm

import os
import numpy as np
import music21 as m21
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation


  Using cached pretty_midi-0.2.10.tar.gz (5.6 MB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.3 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592286 sha256=35b21e9498bc63f37d27445cf20a14038e390937ea6009de95e3a3a7d0ee4216
  Stored in directory: /root/.cache/pip/wheels/a4/f9/9e/08350c27e386558df0ae234e28a8facd145ba45506ddd1b989
Successfully built pretty_midi


In [4]:
# ===============================
# Step 2: Load Dataset
# ===============================
dataset_path = "/root/.cache/kagglehub/datasets/imsparsh/lakh-midi-clean/versions/1"

# Collect all MIDI file paths
midi_files = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(".mid") or file.endswith(".midi"):
            midi_files.append(os.path.join(root, file))

print(f"Total MIDI files found: {len(midi_files)}")


Total MIDI files found: 17232


In [5]:
# ===============================
# Step 3: Preprocess MIDI with music21
# ===============================
notes = []

for file in tqdm(midi_files[:500]):  # Limit to 500 for faster processing
    try:
        midi = m21.converter.parse(file)
        notes_to_parse = None

        parts = m21.instrument.partitionByInstrument(midi)
        if parts:  # File has instrument parts
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, m21.note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, m21.chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    except Exception as e:
        continue

print("Total notes collected:", len(notes))


 10%|▉         | 49/500 [03:07<27:46,  3.70s/it]/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=10, channel=None, data=b'Fl\x84che  '>; getting generic Instrument
  warnings.warn(
 16%|█▌        | 78/500 [05:34<36:05,  5.13s/it]/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=14, channel=None, data=b'\xa9 1976 Union Songs AB'>; getting generic Instrument
  warnings.warn(
 16%|█▋        | 82/500 [05:54<35:35,  5.11s/it]/usr/local/lib/python3.12/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=11, channel=None, data=b'Fl\x84che  '>; getting generic Instrument
  warnings.warn(
 34%|███▎      | 168/500 [12:46<21:52,  3.95s/it]/usr/local/lib

Total notes collected: 118351


In [6]:
# ===============================
# Step 4: Prepare Sequences
# ===============================
sequence_length = 50

# Create vocabulary of notes
pitchnames = sorted(set(notes))
n_vocab = len(pitchnames)

# Map notes to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

for i in range(0, len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in seq_in])
    network_output.append(note_to_int[seq_out])

n_patterns = len(network_input)

network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)  # Normalize
network_output = tf.keras.utils.to_categorical(network_output)

print("Input shape:", network_input.shape)
print("Output shape:", network_output.shape)


Input shape: (118301, 50, 1)
Output shape: (118301, 386)


In [7]:
# ===============================
# Step 5: Build the Model (LSTM)
# ===============================
model = Sequential()
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50, 512)        │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50, 512)        │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50, 512)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 512)            │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 386)            │        99,202 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,481,602 (20.91 MB)

 Trainable params: 5,481,602 (20.91 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# ===============================
# Step 6: Train the Model
# ===============================
history = model.fit(network_input, network_output, epochs=50, batch_size=64)


Epoch 1/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 84s 42ms/step - loss: 4.7026
Epoch 2/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 84s 45ms/step - loss: 4.6073
Epoch 3/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 142s 45ms/step - loss: 4.6022
Epoch 4/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 142s 45ms/step - loss: 4.5988
Epoch 5/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 83s 45ms/step - loss: 4.6002
Epoch 6/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 142s 45ms/step - loss: 4.5935
Epoch 7/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 142s 45ms/step - loss: 4.6000
Epoch 8/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 142s 45ms/step - loss: 4.6003
Epoch 9/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 142s 45ms/step - loss: 4.6048
Epoch 10/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 142s 45ms/step - loss: 4.5901
Epoch 11/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 82s 45ms/step - loss: 4.5980
Epoch 12/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 142s 45ms/step - loss: 4.5990
Epoch 13/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 142s 45ms/step - loss: 4.5988
Epoch 14/50
1849/1849 ━━━━━━━━━━━━━━━━━━━━ 142s 45ms/step - loss

In [13]:
# Save the trained model
model.save("music_gen_model.h5")
print("✅ Model saved successfully!")


✅ Model saved successfully!


In [14]:
from tensorflow.keras.models import load_model

model = load_model("music_gen_model.h5")
print("✅ Model loaded successfully!")


✅ Model loaded successfully!


In [22]:
import pickle

# Save trained model (use .keras instead of .h5 for modern Keras)
model.save("music_gen_model.keras")

# Save note mappings
with open("note_mappings.pkl", "wb") as f:
    pickle.dump(int_to_note, f)

print("✅ Model and mappings saved successfully!")


✅ Model and mappings saved successfully!


In [23]:
# 🎵 Complete Music Generation Pipeline in One Cell

import os, glob, pickle, random
import numpy as np
import music21
from music21 import converter, instrument, note, chord, stream
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
import gradio as gr

# === Step 1: Load MIDI files and Extract Notes ===
midi_path = "/root/.cache/kagglehub/datasets/imsparsh/lakh-midi-clean/versions/1/**/*.mid"

notes = []
for file in glob.glob(midi_path, recursive=True)[:200]:  # limit for speed
    try:
        midi = converter.parse(file)
        parts = instrument.partitionByInstrument(midi)
        elements = parts.parts[0].recurse() if parts else midi.flat.notes
        for element in elements:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append(".".join(str(n) for n in element.normalOrder))
    except Exception as e:
        print(f"⚠️ Skipped {file}: {e}")

print("✅ Total Notes Extracted:", len(notes))

# === Step 2: Prepare Sequences ===
sequence_length = 50
pitchnames = sorted(set(notes))
note_to_int = {note: number for number, note in enumerate(pitchnames)}
int_to_note = {number: note for number, note in enumerate(pitchnames)}

network_input, network_output = [], []
for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append([note_to_int[ch] for ch in seq_in])
    network_output.append(note_to_int[seq_out])

X = np.reshape(network_input, (len(network_input), sequence_length, 1)) / float(len(pitchnames))
y = tf.keras.utils.to_categorical(network_output, num_classes=len(pitchnames))

print("✅ Input Shape:", X.shape, " Output Shape:", y.shape)

# === Step 3: Build Model ===
model = Sequential([
    LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(256, activation="relu"),
    Dropout(0.3),
    Dense(len(pitchnames), activation="softmax")
])
model.compile(loss="categorical_crossentropy", optimizer="adam")

# === Step 4: Train Model ===
model.fit(X, y, epochs=5, batch_size=64)  # keep epochs low for testing

# === Step 5: Save Model + Mappings ===
model.save("music_gen_model.keras")
with open("note_mappings.pkl", "wb") as f:
    pickle.dump(int_to_note, f)
print("✅ Model and mappings saved successfully!")

# === Step 6: Load Model + Mappings ===
model = load_model("music_gen_model.keras")
with open("note_mappings.pkl", "rb") as f:
    int_to_note = pickle.load(f)

# === Step 7: Music Generation Function ===
def generate_music(length=100):
    start = np.random.randint(0, len(network_input)-1)
    pattern = network_input[start]
    prediction_output = []

    for note_index in range(length):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1)) / float(len(pitchnames))
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern.append(index)
        pattern = pattern[1:]

    offset = 0
    output_notes = []
    for pattern in prediction_output:
        if "." in pattern or pattern.isdigit():
            chord_notes = [note.Note(int(n)) for n in pattern.split(".")]
            for n in chord_notes:
                n.storedInstrument = instrument.Piano()
            new_chord = chord.Chord(chord_notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_file = "generated_music.mid"
    midi_stream.write("midi", fp=midi_file)
    return midi_file

# === Step 8: Gradio Interface ===
def gradio_interface(length):
    midi_path = generate_music(length)
    return midi_path

iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Slider(50, 300, value=100, step=10, label="Length of Generated Music (Notes)"),
    outputs="file",
    title="🎶 AI Music Generator",
    description="Generate new music sequences trained on MIDI data!"
)

iface.launch()


⚠️ Skipped /root/.cache/kagglehub/datasets/imsparsh/lakh-midi-clean/versions/1/ABBA/Ive_Been_Waiting_For_You.mid: badly formed midi string: missing leading MTrk
⚠️ Skipped /root/.cache/kagglehub/datasets/imsparsh/lakh-midi-clean/versions/1/ABBA/Voulez_Vous.1.mid: cannot place n between multiples: 4.3168778689674135e+31, 4.3168778689674135e+31
✅ Total Notes Extracted: 42761
✅ Input Shape: (42711, 50, 1)  Output Shape: (42711, 300)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 4.7747
Epoch 2/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - loss: 4.5369
Epoch 3/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 4.2616
Epoch 4/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - loss: 4.1653
Epoch 5/5
668/668 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - loss: 4.0449
✅ Model and mappings saved successfully!
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ce7dcf78a9c1c25d8f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
